In [212]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import os
from datetime import datetime
from functools import partial
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Separating data in train data and validation data

Here, we used stratified train test split sklearn function to separate our data. It's defined only 10% of all data to validation data. Here we have experimented to predict 0-5 handwritten digits.

In [213]:
tf.reset_default_graph()

train_data = pd.read_csv("train.csv")

train_labels = train_data.iloc[:, 0].values
boolean_indices = train_labels < 5 
train_labels = train_labels[boolean_indices]

train_data = train_data.iloc[:, 1:].values
train_data = train_data[boolean_indices]
train_data = train_data.astype(np.float64)

print(train_data.shape)

train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels,
                                                                    test_size=0.1, stratify=train_labels)

print(test_labels.shape)

(21416, 784)
(2142,)


In [214]:
class MNISTClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, layers=[100, 100, 100, 100, 100], batch_size=128, optimizer_class = tf.train.AdamOptimizer,
                 learning_rate=0.01, activation = tf.nn.elu, dropout_rate=None, momentum_batch=None, random_state=None):
        self.layers = layers
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.momentum_batch = momentum_batch
        self.batch_size = batch_size
        self._session = None
        self._training = None
        
        if random_state is not None:
            tf.set_random_seed(random_state)
            np.random.seed(random_state)
        
    def _dnn(self, X):
        '''Here we build our dnn structure'''
        for index, n_layer in enumerate(self.layers):
            if self.dropout_rate:
                X = tf.layers.dropout(X, self.dropout_rate, training=self._training)

            X = tf.layers.dense(X, n_layer)
            if self.momentum_batch:
                X = tf.layers.batch_normalization(X, training=self._training, momentum=self.momentum_batch)
            X = self.activation(X, name="hidden%d" % (index))

        return X

    def _prepare_dataset(self, X_shape = None, y = None):
        
        '''Prepare our tensorflow dataset'''
        self._batch_size = tf.placeholder(tf.int64, shape=(), name="batch_size")
        self._training = tf.placeholder_with_default(False, shape=[], name="training")
        X = tf.placeholder(tf.float64, shape=[None, X_shape[1]], name="X")
        y = tf.placeholder(tf.int64, shape=[None], name="y")

        def map_fn(data, labels):
            data = tf.math.divide(data, 255)
            return data, labels
        
        dataset = tf.data.Dataset.from_tensor_slices((X, y))
        if self._training is not None:
            dataset = dataset.repeat().shuffle(X_shape[0])

        dataset = dataset.batch(self._batch_size)
        dataset = dataset.map(map_fn, num_parallel_calls = tf.data.experimental.AUTOTUNE)
        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

        self._iterator = dataset.make_initializable_iterator()

        self._data, self._labels = self._iterator.get_next()
        self._X = X
        self._y = y


    def _build_graph(self, n_outputs):
        '''Building tensorflow graph'''

        with tf.name_scope("build"):

            last_hidden_layer = self._dnn(self._data)

            logits = tf.layers.dense(last_hidden_layer, n_outputs, name="outputs")
            logits = tf.cast(logits, tf.float32)
            y_proba = tf.nn.softmax(logits, name="y_proba")

        with tf.name_scope("loss"):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self._labels, logits=logits)
            loss = tf.reduce_mean(xentropy, name="loss")
            loss_summary = tf.summary.scalar("log_loss", loss)

        with tf.name_scope("train"):
            optimizer = self.optimizer_class(self.learning_rate)
            training_op = optimizer.minimize(loss)

        with tf.name_scope("eval"):
            correct = tf.nn.in_top_k(logits, self._labels, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float64), name="accuracy")
            accuracy_summary = tf.summary.scalar("log_accuracy", accuracy)

        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
            saver = tf.train.Saver()

        self._saver = saver
        self._accuracy, self._accuracy_summary = accuracy, accuracy_summary
        self._loss, self._loss_summary = loss, loss_summary
        self._training_op = training_op
        self._y_proba = y_proba
        self._init = init

    
    def _get_model_params(self):
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
            return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}
    
    def _restore_model_params(self, model_params):
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict = feed_dict)
        
        
    def fit(self, X, y, n_epochs=10001, summary=False):

        def logdir(prefix=""):
            now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
            root_dir = "tf_logs/"
            return root_dir + prefix + "run-" + now
        
        
        n_inputs = X.shape
        self._classes = np.unique(y)
        n_outputs = len(self._classes)

        self._graph = tf.Graph()
        with self._graph.as_default():
            self._prepare_dataset(n_inputs)
            self._build_graph(n_outputs)
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            if summary:
                file_writer = tf.summary.FileWriter(logdir("MNIST"), tf.get_default_graph())

                
        best_loss = np.infty
        counter_early_stopping=0
        counter_max_value = 200

            
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:               #dúvida                                         
            sess.run(self._iterator.initializer, feed_dict={self._X: X, self._y: y,
                                                            self._training: True, self._batch_size : self.batch_size})

#                 if os.path.isfile(checkpoint_epoch_path):
#                     with open(checkpoint_epoch_path, "rb") as f:
#                         start_epoch = int(f.read())
#                     print("Foi interrompido! Parou no {} epoch".format(start_epoch))
#                     saver.restore(sess, checkpoint_path)

#                 else:
            start_epoch = 0
            sess.run(self._init)

            for epoch in range(start_epoch, n_epochs):
                _, _loss_summary, _accuracy_summary, loss_value, accuracy_value = sess.run([self._training_op,
                                                                                            self._loss_summary,
                                                                                            self._accuracy_summary,
                                                                                            self._loss, self._accuracy])
                
                if extra_update_ops:
                    sess.run(extra_update_ops)
                
                if loss_value < best_loss:
                    best_params = self._get_model_params()
                    best_loss = loss_value
                    counter_early_stopping = 0
                    
                else:
                    counter_early_stopping += 1
                    if counter_early_stopping > counter_max_value:
                        print("Early Stopping!")
                        break

                if epoch % 10 == 0:
                    if summary:
                        file_writer.add_summary(_loss_summary, epoch)
                        file_writer.add_summary(_accuracy_summary, epoch)

                if epoch % 50 == 0:
                    accuracy_value = sess.run(self._accuracy)
                    print("Best Loss: {:.6f}\t\tTrain Score: {:.4f}".format(best_loss, accuracy_value))

            accuracy_train = sess.run(self._accuracy)
            
            if best_params:
                self._best_params = best_params
                self._restore_model_params(self._best_params)
                
        return self
    
    def score(self, X, y, method = accuracy_score):
        with self._session.as_default() as sess:
            self._restore_model_params(self._best_params)
            sess.run(self._iterator.initializer, feed_dict={self._X: X, self._y: y, self._batch_size: len(y),
                                                            self._training: None})

            _, y_proba, labels_val = sess.run([self._loss, self._y_proba, self._labels])
            y_pred = np.argmax(y_proba, axis = 1)
            
            return method(labels_val, y_pred)
            
    def predict(self, X, y):
         with self._session.as_default() as sess:
            self._restore_model_params(self._best_params)
            sess.run(self._iterator.initializer, feed_dict={self._X: X, self._y: y, self._batch_size: len(X),
                                                            self._training: None})

            y_proba = self._y_proba.eval()
            class_indices = np.argmax(y_proba, axis = 1)
            
            return np.array([[self._classes[class_index]]
                         for class_index in class_indices], np.int32)
            
        
    def save(self, path):
        self._saver.save(self._session, path)

In [215]:
# mnist = MNISTClassifier(random_state=42)

# mnist.fit(train_data, train_labels)
# accuracy = mnist.score(test_data, test_labels)
# # precision = mnist.score(test_data, test_labels, method = precision_score)
# print("Test accuracy score: {}".format(accuracy))
# # print("Test precision score: {}".format(precision))

In [216]:
# param_distribs = {
#     "layers": [[100, 100, 100, 100, 100], [512, 256, 128, 64, 32], [50, 50, 50, 50, 50]],
#     "batch_size": [16, 32, 64, 128, 256],
#     "learning_rate": [0.01, 0.02, 0.05, 0.1, 0.5],
#     "activation": [tf.nn.relu, tf.nn.elu],
#     "momentum_batch": [None, 0.9, 0.99, 0.999, 0.9999],
#     "dropout_rate": [None, 0.1, 0.2, 0.3, 0.4, 0.5],
#     "optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)]
# }

# clf = RandomizedSearchCV(MNISTClassifier(random_state=42), param_distribs, cv=3, n_iter=100, random_state=42)
# clf.fit(train_data, train_labels)

In [217]:
mnist = MNISTClassifier(momentum_batch=0.999, learning_rate=0.02, layers=[50, 50, 50, 50, 50], dropout_rate=0.2, batch_size=256, random_state=42)
mnist.fit(train_data, train_labels)
# accuracy = mnist.score(test_data, test_labels)
# print(accuracy)

Best Loss: 1.628094		Train Score: 0.4844
Best Loss: 0.072601		Train Score: 0.9688
Best Loss: 0.036623		Train Score: 0.9883
Best Loss: 0.011734		Train Score: 0.9766
Best Loss: 0.011734		Train Score: 0.9844
Best Loss: 0.011734		Train Score: 0.9766
Best Loss: 0.006813		Train Score: 0.9883
Best Loss: 0.006563		Train Score: 0.9727
Best Loss: 0.001591		Train Score: 0.9961
Best Loss: 0.001591		Train Score: 0.9922
Best Loss: 0.001591		Train Score: 0.9961
Best Loss: 0.000490		Train Score: 0.9922
Best Loss: 0.000490		Train Score: 0.9883
Best Loss: 0.000490		Train Score: 0.9883
Best Loss: 0.000490		Train Score: 1.0000
Best Loss: 0.000445		Train Score: 0.9961
Best Loss: 0.000445		Train Score: 0.9961
Best Loss: 0.000445		Train Score: 0.9961
Best Loss: 0.000445		Train Score: 1.0000
Early Stopping!


MNISTClassifier(activation=<function elu at 0x00000225B36F2E18>,
        batch_size=256, dropout_rate=0.2, layers=[50, 50, 50, 50, 50],
        learning_rate=0.02, momentum_batch=0.999,
        optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
        random_state=None)

In [218]:
indices = mnist.predict(train_data[10:14][:], train_labels[10:14])
print(train_labels[10:14])
print(indices)
# accuracy_score(train_labels, y_pred)

InvalidArgumentError: You must feed a value for placeholder tensor 'y' with dtype int64 and shape [?]
	 [[node y (defined at <ipython-input-214-a08665d1c13e>:37) ]]

Caused by op 'y', defined at:
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-package